In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

In [108]:
#!pip install nltk

In [109]:
df = pd.read_csv('../Dataset/emotion_dataset.csv')
df

,text,label
0,i feel awful about it too because it s my job ...,0
1,im alone i feel awful,0
2,ive probably mentioned this before but i reall...,1
3,i was feeling a little low few days back,0
4,i beleive that i am much more sensitive to oth...,2
...,...,...
416804,that was what i felt when i was finally accept...,1
416805,i take every day as it comes i m just focussin...,4
416806,i just suddenly feel that everything was fake,0
416807,im feeling more eager than ever to claw back w...,1


In [110]:
df = df[0:20000]
column= 'text'


In [111]:
# preprocessing

df[column] = df[column].str.lower()

STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

df[column] = df[column].apply(lambda x: remove_stopwords(x))

def remove_splcharacters(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = re.sub('\s+', ' ', text)
    return text

df[column] = df[column].apply(lambda x: remove_splcharacters(x))

df.head()

C:\Users\ANISH\AppData\Local\Temp\ipykernel_25752\2706339257.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].str.lower()
C:\Users\ANISH\AppData\Local\Temp\ipykernel_25752\2706339257.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: remove_stopwords(x))
C:\Users\ANISH\AppData\Local\Temp\ipykernel_25752\2706339257.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,text,label
0,feel awful job get position succeed happen,0
1,im alone feel awful,0
2,ive probably mentioned really feel proud actua...,1
3,feeling little low days back,0
4,beleive much sensitive peoples feelings tend c...,2


In [112]:
# Lemmatization

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word, pos ='v') for word in text.split()])

df[column] = df[column].apply(lambda x: lemmatize_text(x))

C:\Users\ANISH\AppData\Local\Temp\ipykernel_25752\3103777441.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(lambda x: lemmatize_text(x))


In [113]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df[column])


import pickle
# Save tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


word_index = tokenizer.word_index
vocab_size = len(word_index)
vocab_size

13737

In [114]:
# padding the data
sequence = tokenizer.texts_to_sequences(df[column])

# maximum length of the data
max_len = 40
padded_seq = pad_sequences(sequence,maxlen=max_len, padding='post', truncating= 'post')
padded_seq

array([[   1,  367,  201, ...,    0,    0,    0],
       [   3,  117,    1, ...,    0,    0,    0],
       [  22,  214,  514, ...,    0,    0,    0],
       ...,
       [   1, 1487,  448, ...,    0,    0,    0],
       [ 138,    1,   17, ...,    0,    0,    0],
       [   1,   26,  110, ...,    0,    0,    0]], dtype=int32)

In [115]:
# Word Embedding

#create embedding index
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        embed_list = np.array(values[1:]).astype('float32')
        embedding_index[word]= embed_list



In [116]:
#create embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size+1, embedding_dim))

for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [117]:
# Split dataset
from sklearn.model_selection import train_test_split

xtrain,  xtest, ytrain,ytest = train_test_split(padded_seq,df['label'], train_size=0.9, random_state=42)

from keras.utils import to_categorical

ytrain = to_categorical(ytrain)

In [118]:
# Build the model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(
    Embedding(
        input_dim=vocab_size + 1,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=False,  # freeze pretrained embeddings
    )
)
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │     1,373,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,373,800 (5.24 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,373,800 (5.24 MB)

In [119]:

model.fit(xtrain,ytrain,validation_split=0.1,epochs=30,batch_size=32)

Epoch 1/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.3343 - loss: 1.6103 - val_accuracy: 0.3278 - val_loss: 1.5923
Epoch 2/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.3336 - loss: 1.5687 - val_accuracy: 0.3267 - val_loss: 1.5989
Epoch 3/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.3318 - loss: 1.5728 - val_accuracy: 0.3267 - val_loss: 1.5918
Epoch 4/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3407 - loss: 1.5698 - val_accuracy: 0.3267 - val_loss: 1.5941
Epoch 5/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.3337 - loss: 1.5743 - val_accuracy: 0.3267 - val_loss: 1.5917
Epoch 6/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.3393 - loss: 1.5737 - val_accuracy: 0.3267 - val_loss: 1.5933
Epoch 7/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.3313 - loss: 1.5730 - val_accuracy: 0.3267 - val_loss: 1.5935
Epoch 8/30
507/507 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.3519 - loss: 1.5711 - val_accu

In [ ]:
predict = model.predict(xtest)
predict = np.argmax(predict,axis=1)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [121]:
from sklearn.metrics import classification_report
print(classification_report(ytest, predict))

              precision    recall  f1-score   support

           0       0.91      0.96      0.94       610
           1       0.96      0.88      0.92       639
           2       0.76      0.81      0.78       168
           3       0.90      0.90      0.90       271
           4       0.86      0.93      0.89       240
           5       0.89      0.76      0.82        72

    accuracy                           0.90      2000
   macro avg       0.88      0.87      0.88      2000
weighted avg       0.91      0.90      0.90      2000



In [122]:
model.save('emotion_analyzer.h5')

In [123]:
inp = [ 441, 3328,    1, 1179,  763,   17,    0,    0,    0,    0,    0,
          0,    0,    1111110,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0]
inp = np.expand_dims(inp, axis=0).astype('int32')
model.predict(inp)



InvalidArgumentError: Graph execution error:

Detected at node sequential_9_1/embedding_9_1/GatherV2 defined at (most recent call last):
  File "c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\runpy.py", line 86, in _run_code

  File "c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Nishanthi\Hope_AI\Projects\Chat_Emotion_Detection\venv\lib\asyncio\base_events.py", line 60

In [92]:
inp

array([[ 441, 3328,    1, 1179,  763,   17,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int32)